Übung Horizontal Pod Autoscaler
============================

![](https://d33wubrfki0l68.cloudfront.net/4fe1ef7265a93f5f564bd3fbb0269ebd10b73b4e/1775d/images/docs/horizontal-pod-autoscaler.svg)

Quelle [Horizontal Pod Autoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale/)
- - -

Der horizontale Pod-Autoscaler skaliert automatisch die Anzahl der Pods in einem ReplicaSet, einem Deployment oder einem StatefulSet.

Die Skalierung kann anhand der beobachteten CPU-Auslastung oder mittels benutzerdefinierten Metriken erfolgen. Für Details siehe [hier](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale/#algorithm-details).

Die Metriken kommen von einem separat gestarteten [Metric Server](https://github.com/kubernetes-sigs/metrics-server).

In [ ]:
! kubectl get pods --namespace metrics

***
Test Image 
---------------

Um den horizontalen Pod-Autoscaler zu demonstrieren, verwenden wir ein benutzerdefiniertes Docker-Image, das auf dem PHP-Apache-Image basiert. Die Docker-Datei hat folgenden Inhalt:

    FROM php:5-apache
    COPY index.php /var/www/html/index.php
    RUN chmod a+rx index.php
    
Es definiert eine index.php-Seite, die einige CPU-intensive Berechnungen durchführt:

    <?php
      $x = 0.0001;
      for ($i = 0; $i <= 1000000; $i++) {
        $x += sqrt($x);
      }
      echo "OK!";
    ?>
    
Zunächst starten wir den Pod, in der das Image ausgeführt wird, und stellen es als Dienst bereit:    

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Namespace
metadata:
  name: hpa
%EOF%

cat <<%EOF% | kubectl apply --namespace hpa -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: php-apache
spec:
  selector:
    matchLabels:
      run: php-apache
  replicas: 1
  template:
    metadata:
      labels:
        run: php-apache
    spec:
      containers:
      - name: php-apache
        image: k8s.gcr.io/hpa-example
        ports:
        - containerPort: 80
        resources:
          limits:
            cpu: 500m
          requests:
            cpu: 200m
---
apiVersion: v1
kind: Service
metadata:
  name: php-apache
  labels:
    run: php-apache
spec:
  ports:
  - port: 80
  selector:
    run: php-apache
%EOF%


### Erstellung horizontalen Pod-Autoscaler

Nachdem der Server ausgeführt wurde, erstellen wir den Autoscaler. 

Mit dem folgenden Befehl wird ein horizontaler Pod-Autoscaler erstellt, der zwischen 1 und 10 Instanzen des obigen Pods verwaltet.

Grob gesagt wird der HPA die Anzahl der Replikate (via Deployment) erhöhen und verringern, um eine durchschnittliche CPU-Auslastung über alle Pods von 50% aufrechtzuerhalten. 

In [ ]:
! kubectl autoscale deployment php-apache --namespace hpa --cpu-percent=50 --min=1 --max=10

Wir schauen uns den aktuellen Status unserer Ressourcen an:

In [ ]:
! kubectl get pods --namespace hpa --show-kind=true
! kubectl get hpa  --namespace hpa --show-kind=true 

### Last erhöhen

Nun werden wir sehen, wie der Autoscaler auf erhöhte Last reagiert. Wir werden einen Container starten und eine Endlosschleife von Abfragen an den PHP-Apache-Dienst senden.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace hpa -f -
apiVersion: v1
kind: Pod
metadata:
  labels:
    app.kubernetes.io/name: load-generator
  name: load-generator
spec:
  containers:
  - image: busybox
    name: busybox
    command: [ "/bin/sh", "-c", "--" ]
    args: [ "while true; do wget -q -O- http://php-apache; done" ]
%EOF%


Beim Horizontal Pod Autoscaler sollten wir jetzt eine Last > 50% sehen. Die anzeigte `Last / 50` ergibt die Anzahl Pods die am Schluss laufen müssen.

In [ ]:
! kubectl get pods --namespace hpa --show-kind=true
! kubectl get hpa  --namespace hpa --show-kind=true 

Wird der Pods `load-generator` gelöscht, verringert sich die Last und die Anzahl `php-apache` Pods werden wieder zurückgefahren.

In [ ]:
! kubectl delete --namespace hpa pod/load-generator

In [ ]:
! kubectl get all,hpa --namespace hpa 

***
Aufräumen

In [ ]:
! kubectl delete namespace hpa